## Regression: What are the drivers of worker productivity?
For this project, we will aim to predict the level of productivity for workers in the garment industry given a variety of production and supply chain metrics. To do this, we will use  data collected from January to March 2015 by the Industrial Engineering (IE) department of a garment manufacturing unit in
Bangladesh. This kind of model would have all sorts of real world applications within the fields of operational research, labour economics, and supply chain management.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Data set overview

To start out, lets get an overview of our data set dimensions and feature types. We'll be doing a very simple overview in this section but feel free to perform more extensive EDA on your own: the more you know about your data, the better you can harness it through modelling.

Start out with the following tasks:
- Read in the data set as a pandas DataFrame (file path: `data/garments_worker_productivity.csv'`)
- Output the first ten data set rows
- Output the last ten data set rows
- Print out a comprehensive summary of the data set (dimensions, variable names, data types, and null values)

In [2]:
# Read  data set
data = pd.read_csv('data/garments_worker_productivity.csv')

In [3]:
# Output the first ten rows
data.head(10)

,date,quarter,department,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
0,1/1/2015,Quarter1,sweing,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725
1,1/1/2015,Quarter1,finishing,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500
2,1/1/2015,Quarter1,sweing,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
3,1/1/2015,Quarter1,sweing,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570
4,1/1/2015,Quarter1,sweing,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382
5,1/1/2015,Quarter1,sweing,7,0.80,25.90,984.0,6720,38,0.0,0,0,56.0,0.800125
6,1/1/2015,Quarter1,finishing,2,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.755167
7,1/1/2015,Quarter1,sweing,3,0.75,28.08,795.0,6900,45,0.0,0,0,57.5,0.753683
8,1/1/2015,Quarter1,sweing,2,0.75,19.87,733.0,6000,34,0.0,0,0,55.0,0.753098
9,1/1/2015,Quarter1,sweing,1,0.75,28.08,681.0,6900,45,0.0,0,0,57.5,0.750428


In [4]:
# Output the last ten rows
data.tail(10)

,date,quarter,department,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity
1187,3/11/2015,Quarter2,sweing,4,0.75,26.82,1054.0,7080,45,0.0,0,0,59.0,0.750051
1188,3/11/2015,Quarter2,sweing,5,0.70,26.82,992.0,6960,30,0.0,0,1,58.0,0.700557
1189,3/11/2015,Quarter2,sweing,8,0.70,30.48,914.0,6840,30,0.0,0,1,57.0,0.700505
1190,3/11/2015,Quarter2,sweing,6,0.70,23.41,1128.0,4560,40,0.0,0,1,38.0,0.700246
1191,3/11/2015,Quarter2,sweing,7,0.65,30.48,935.0,6840,26,0.0,0,1,57.0,0.650596
1192,3/11/2015,Quarter2,finishing,10,0.75,2.90,NaN,960,0,0.0,0,0,8.0,0.628333
1193,3/11/2015,Quarter2,finishing,8,0.70,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1194,3/11/2015,Quarter2,finishing,7,0.65,3.90,NaN,960,0,0.0,0,0,8.0,0.625625
1195,3/11/2015,Quarter2,finishing,9,0.75,2.90,NaN,1800,0,0.0,0,0,15.0,0.505889
1196,3/11/2015,Quarter2,finishing,6,0.70,2.90,NaN,720,0,0.0,0,0,6.0,0.394722


In [5]:
# Print the data set overview (dimensions, variable names, data types, null values)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   date                   1197 non-null   object 
 1   quarter                1197 non-null   object 
 2   department             1197 non-null   object 
 3   team                   1197 non-null   int64  
 4   targeted_productivity  1197 non-null   float64
 5   smv                    1197 non-null   float64
 6   wip                    691 non-null    float64
 7   over_time              1197 non-null   int64  
 8   incentive              1197 non-null   int64  
 9   idle_time              1197 non-null   float64
 10  idle_men               1197 non-null   int64  
 11  no_of_style_change     1197 non-null   int64  
 12  no_of_workers          1197 non-null   float64
 13  actual_productivity    1197 non-null   float64
dtypes: float64(6), int64(5), object(3)
memory usage: 131.0+ 

### Feature engineering with dates

One of the features in our data set provides date information. In this format, the feature can be quite useless, but there are several things that we can do to extract useful information from it relating to worker productivity. To start, we want to convert the feature entries into `datetime` objects. To do this:
- Use the `pd.to_datetime()` function to reset the `'date'` column in our data set.
> **Note:** In order to get the right format, pass the argument `format = '%m/%d/%Y' `, and have a read [here](https://www.w3schools.com/python/python_datetime.asp) for further information on how to handle datetime objects.

In [6]:
# Converting 'date' column into datetime 
data['date'] = pd.to_datetime(data['date'], format = '%m/%d/%Y' ) 

With our 'date' column in `datetime` format, we can now proceed to extract a number of additional features. Although you can find the full list of `datetime` attributes [here](https://pandas.pydata.org/pandas-docs/version/0.23/api.html#datetimelike-properties), we will focus on extracting the day of the week and the month as separate features. To extract a `datetime` attribute as a feature, we can use the following command:
- `data['<new_feature_label>'] = data['date'].dt.<attribute>`

Try extracting the attributes `.month` and `.dayofweek` and assigning it to columns with the labels `'month'` and `'day'` respectively:

In [7]:
# Creating month and day of the week features
data['month'] = data['date'].dt.month
data['day'] = data['date'].dt.dayofweek

Lastly, we want to drop the `'date'` feature from our data set (as this can be hard to encode) and output the first ten rows of our data set and the data set overview to check the new features:

In [8]:
# dropping 'date' column
data = data.drop(columns='date')

In [9]:
# Output the first ten rows
data.head(10)

,quarter,department,team,targeted_productivity,smv,wip,over_time,incentive,idle_time,idle_men,no_of_style_change,no_of_workers,actual_productivity,month,day
0,Quarter1,sweing,8,0.80,26.16,1108.0,7080,98,0.0,0,0,59.0,0.940725,1,3
1,Quarter1,finishing,1,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.886500,1,3
2,Quarter1,sweing,11,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570,1,3
3,Quarter1,sweing,12,0.80,11.41,968.0,3660,50,0.0,0,0,30.5,0.800570,1,3
4,Quarter1,sweing,6,0.80,25.90,1170.0,1920,50,0.0,0,0,56.0,0.800382,1,3
5,Quarter1,sweing,7,0.80,25.90,984.0,6720,38,0.0,0,0,56.0,0.800125,1,3
6,Quarter1,finishing,2,0.75,3.94,NaN,960,0,0.0,0,0,8.0,0.755167,1,3
7,Quarter1,sweing,3,0.75,28.08,795.0,6900,45,0.0,0,0,57.5,0.753683,1,3
8,Quarter1,sweing,2,0.75,19.87,733.0,6000,34,0.0,0,0,55.0,0.753098,1,3
9,Quarter1,sweing,1,0.75,28.08,681.0,6900,45,0.0,0,0,57.5,0.750428,1,3


In [10]:
# Print the data set overview (dimensions, variable names, data types, null values)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   quarter                1197 non-null   object 
 1   department             1197 non-null   object 
 2   team                   1197 non-null   int64  
 3   targeted_productivity  1197 non-null   float64
 4   smv                    1197 non-null   float64
 5   wip                    691 non-null    float64
 6   over_time              1197 non-null   int64  
 7   incentive              1197 non-null   int64  
 8   idle_time              1197 non-null   float64
 9   idle_men               1197 non-null   int64  
 10  no_of_style_change     1197 non-null   int64  
 11  no_of_workers          1197 non-null   float64
 12  actual_productivity    1197 non-null   float64
 13  month                  1197 non-null   int64  
 14  day                    1197 non-null   int64  
dtypes: f

### Dummy encoding

Now that we've extracted some new features from our date data, we want to encode all our features so that they're ready for machine learning modeling. This includes encoding all of our categorical features as 'dummy variables'. Whilst this may seem easy at first sight, check the unique values (using `.unique()`) for the following features: `'team'`, `'month'`, `'day'`. Do these seem like continuous or categorical variables to you?

In [11]:
# Checking unique values for 'team'
data['team'].unique()

array([ 8,  1, 11, 12,  6,  7,  2,  3,  9, 10,  5,  4], dtype=int64)

In [12]:
# Checking unique values for 'month' 
data['month'].unique()

array([1, 2, 3], dtype=int64)

In [13]:
# Checking unique values for 'day' 
data['day'].unique()

array([3, 5, 6, 0, 1, 2], dtype=int64)

As we can see, these features, although of numerical data type, fall much better under the definition of categorical variables. Had we not noticed, they would not get dummy encoded by the `pd.get_dummies()` function, leading to the incorrect handling of our data. Thus, its important to always check what kind of features we have, irrespective if data type. We can get around this encoding problem by casting these features as `str` types and then proceeding with the dummy encoding. In our data set:
- Cast the features `'team'`, `'month'`, and `'day'` as strings.
- Use the `pd.get_dummies()` function to dummy encode categorical variables (Hint: remember to drop one dummy pre categorical feature using the `drop_first` parameter).

In [14]:
# Cast 'team', 'month', and 'day' features as str
data['team'] = data['team'].astype(str)
data['month'] = data['month'].astype(str)
data['day'] = data['day'].astype(str)

In [15]:
# Generate dummy encoding
# We drop the first categorical variable as the machine learning algorithm will see it the same, but this way we reduced the dimensions of the dataset
data = pd.get_dummies(data, drop_first=True)

In [16]:
# Print data set overview
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1197 entries, 0 to 1196
Data columns (total 34 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   targeted_productivity  1197 non-null   float64
 1   smv                    1197 non-null   float64
 2   wip                    691 non-null    float64
 3   over_time              1197 non-null   int64  
 4   incentive              1197 non-null   int64  
 5   idle_time              1197 non-null   float64
 6   idle_men               1197 non-null   int64  
 7   no_of_style_change     1197 non-null   int64  
 8   no_of_workers          1197 non-null   float64
 9   actual_productivity    1197 non-null   float64
 10  quarter_Quarter2       1197 non-null   uint8  
 11  quarter_Quarter3       1197 non-null   uint8  
 12  quarter_Quarter4       1197 non-null   uint8  
 13  quarter_Quarter5       1197 non-null   uint8  
 14  department_finishing   1197 non-null   uint8  
 15  depa

### Our ML setup
Now that we have our variables encoded we want to set up our data in the usual machine learning configuration: a training and a test set, each with a feature matrix and a target vector. To start:

- Import the `train_test_split()` function from scikit learn
- Declare a target vector y (corresponding to the `actual_productivity` column in the data set)
- Declare a feature matrix X (including all columns except `actual_productivity`)
- Create a training and test set
- Print the dimensions of X and y in both the training and test set.

In [17]:
# Importing train-test split function
from sklearn.model_selection import train_test_split

In [18]:
# Declare the training vector y which has the column label 'actual_productivity'
y = data['actual_productivity']

In [19]:
# Declare feature matrix 
X = data.drop(columns='actual_productivity')

In [20]:
# Create a train-test- split using random state 253, test_size 30%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=253)

In [21]:
# Print dimensions of the training set
print('X_train dimensions: ', X_train.shape)
print('y_train dimensions: ', y_train.shape)

X_train dimensions:  (837, 33)
y_train dimensions:  (837,)


In [22]:
# Print dimensions of the test set
print('X_test dimensions: ', X_test.shape)
print('y_test dimensions: ', y_test.shape)

X_test dimensions:  (360, 33)
y_test dimensions:  (360,)


### Missing value imputation

As you probably already noticed, we have several missing values in the `'wip'` column of our data set. This time, however, we will use a more advanced method to impute missing values. More specifically, we will use the K Nearest Neighbors algorithm to impute values that are similar to other nearby samples, which should hopefully give us more accuracy. To do this:
- Import the `KNNImputer` class from scikit learn
- Create `KNNImputer` transformer, passing `n_neighbors=7` as an argument.
- Fit the transformer using `X_train`
- Impute missing values by calling the `.transform()` method with `X_train` as an argument. Assign this to a new variable `X_full`.

In [33]:
# Importing KNNImputer
from sklearn.impute import KNNImputer

In [37]:
# Creating KNNImputer transformer
kkni = KNNImputer(n_neighbors = 7)

In [39]:
# Fitting and transforming training feature matrix using our imput
kkni.fit(X_train)

KNNImputer(n_neighbors=7)

### Pipelines

You might've noticed that most machine learning with scikit learn involves:
- Instantiating a transformer or model.
- Fitting this to our data using `.fit()`.
- Transforming or predicting using `.transform()` or `.predict()`.

This simple, general process means that we can simplify our ML process using **Pipelines**. A Pipeline allows you to establish a sequence of transformations terminating with *one estimator*. Pipelines can then be treated as any other scikit learn estimator, but each time they perform all of the interim transformation steps in the sequence. This has the benefit of greatly simplifying your code and can also help prevent [data leakage](https://machinelearningmastery.com/data-leakage-machine-learning/). 

To instantiate a Pipeline you must pass as an argument a list of $n+1$ tuples (where $n$ is the number of transforms) in the form: `[('transformer_1_name', transformer_1), ..., ('estimator_name', estimator)]`.

We will now create Pipelines for both an SVM and a Linear Regression model below:
- Import the `StandardScaler`, `SVR`, and `LinearRegression` classes from scikit learn.
- Import the `Pipeline` class from scikit learn.
- For each one of the two models, create a Pipeline with the following transforms (in order): imputation, scaling, estimator.

In [26]:
# Importing StandardScaler
from sklearn.preprocessing import StandardScaler

In [27]:
# Importing KNN, SVM, and Linear Regression models 
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression


In [28]:
# Importing Pipeline
from sklearn.pipeline import Pipeline

In [73]:
# Combine these into two Pipelines, one for each model
lin_reg_pipe = Pipeline([('Imputation', KNNImputer(n_neighbors=7)), 
                         ('Scaler', StandardScaler()), 
                         ('Model', LinearRegression())])

#SVR pipe
svm_pipe = Pipeline([('Imputation', KNNImputer(n_neighbors=7)), 
                         ('Scaler', StandardScaler()), 
                         ('Model', SVR())])


Now we want to see how well our models stack up! For regression problems, one metric that is often used to assess model performance is the **Root Mean Squared Error (RMSE)**. this is given by:
$$\text{RMSE}=\sqrt{\frac{\sum_{i=1}^n (\,y_i\,-\,y_i^{pred}\,)^2}{n}}$$

We can compare our models by computing the cross-validated RMSE scores:
- Import the `cross_val_score()` function from scikit learn.
- Calculate the cross-validated scores for each pipeline estimator. 
- Print the mean score for each pipeline estimator.
> **Note:**  Pass `scoring = 'neg_root_mean_squared_error'` as an argument to `cross_val_score()` to ensure that it computes the RMSE for all models.

In [74]:
from sklearn.model_selection import cross_val_score

In [75]:
# Calculate the cross-validated score of these two models
#cross validation
lin_reg_score = cross_val_score(lin_reg_pipe, X_train, y_train, scoring = 'neg_root_mean_squared_error').mean()
svm_score = cross_val_score(svm_pipe, X_train, y_train, scoring = 'neg_root_mean_squared_error').mean() 

print(svm_score)
print(lin_reg_score)



-0.15265815212350917
-0.1584394088538033


In [76]:
print('Linear Regression RMSE Score:')
print((-1)*lin_reg_score)

Linear Regression RMSE Score:
0.1584394088538033


In [77]:
print('SVM RMSE Score:')
print((-1)*svm_score)

SVM RMSE Score:
0.15265815212350917


### Regularization

We will now learn about a more advanced technique called **regularization** that can help us build better regression models. Regularization is an adaptation on the standard linear regression and essentially works by placing a penalty on the size of the regression coefficients when fitting. This is because large coefficients tend to overfit our model to the training data, so by shrinking them we enable the model to generalize better and improve performance on unseen data. A more detailed description of regularization can be found [here](https://towardsdatascience.com/regularization-in-machine-learning-76441ddcf99a). 

We will focus on one particular kind of regularized regression model known as **Ridge regression**. One key aspect of ridge regression is the `alpha` hyperparameter, which is used to denote the strength of the penalty on coefficient size. A larger value of`alpha` will seek to shrink coefficients more and viseversa. Finding the optimal `alpha` value can be cumbersome, but luckily we can use the hyperparameter optimization techniques learned in Session Five to do this. 

Lets try out using a Ridge model for our productivity problem:
- Import the `Ridge` and `GridSearchCV` classes from scikit learn
- Create a Pipeline with the following transforms (in order): imputation, scaling, Ridge estimator.
- Create a `GridSearchCV` object by passing our Ridge pipeline, and the dictionary `params`.
- Fit the `GridSearchCV` object using `X_train` and `y_train`.
- Print the optimal `alpha` value using the `.best_params_` attribute on our `GridSearchCV` object.
- Print the best RMSE score using the `.best_score_` attribute on our `GridSearchCV` object.
- Assign the optimized model to `ridge_pipe` using the `.best_estimator_` attribute on our `GridSearchCV` object.
> **Note:**  Pass `scoring = 'neg_root_mean_squared_error'` as an argument to `GridSearchCV()` to ensure that it computes the RMSE for our Ridge model.

In [78]:
# Importing Ridge
from sklearn.linear_model import Ridge

In [79]:
# Importing GridSearchCV
from sklearn.model_selection import GridSearchCV

In [80]:
# Creating Ridge Pipeline
ridge_pipe = Pipeline([('Imputation', KNNImputer(n_neighbors=7)), 
                         ('Scaler', StandardScaler()), 
                         ('Model', Ridge())])

In [81]:
params = {'Model__alpha':np.linspace(0, 1, 40)}

In [82]:
# Optimizing alpha hyperparameter using GridSearchCV
grid_search = GridSearchCV(ridge_pipe, params, scoring = 'neg_root_mean_squared_error') #scoring is important here
grid_search.fit(X_train, y_train)
print('Best Alpha: ', grid_search.best_params_)
print('RMSE: ', grid_search.best_score_)

Best Alpha:  {'Model__alpha': 0.0}
RMSE:  -0.1584394088538034


In [83]:

# Retrieving optimized Ridge model
ridge_pipe = grid_search.best_estimator_

### Wrapping it all up

We're nearly there! Now its time to train our models using the whole initial training set to then predict on the test set:
- Import the `mean_squared_error()` function from scikit learn
- Fit each model on the training set.
- Predict with each model on the test set.
- Print out the RMSE of your predictions using mean_squared_error(). 

> **Note**: To get the RMSE instead of the MSE, just pass `squared=False` as an argument.

In [86]:
# Importing mean_squared_error
from sklearn.metrics import mean_squared_error

In [87]:
# Fit and predict Linear Regression pipeline
lin_reg_pipe.fit(X_train, y_train)
y_pred = lin_reg_pipe.predict(X_test)
print('Linear Regression RMSE:')
print(mean_squared_error(y_test, y_pred, squared=False))

Linear Regression RMSE:
0.15584585558454092


In [88]:
# Fit and predict SVM pipeline
svm_pipe.fit(X_train, y_train)
y_pred = svm_pipe.predict(X_test)
print('SVM RMSE:')
print(mean_squared_error(y_test, y_pred, squared=False))

SVM RMSE:
0.1583052933346432


In [89]:
# Fit and predict Ridge pipeline
ridge_pipe.fit(X_train, y_train)
y_pred = ridge_pipe.predict(X_test)
print('Ridge RMSE:')
print(mean_squared_error(y_test, y_pred, squared=False))

Ridge RMSE:
0.1558458555845411
